<a href="https://colab.research.google.com/github/jasmine400/cybersecurity-attack-prediction-with-deep-learninng/blob/main/pre_process_step_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Apr  7 12:02:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
#check if gpu is on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [ ]:
#clear memory
import torch, gc

gc.collect()
torch.cuda.empty_cache()

!nvidia-smi

Thu Apr  7 12:02:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    27W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install OTXv2

In [ ]:
from OTXv2 import OTXv2, IndicatorTypes

In [ ]:
import pandas
from pandas.io.json import json_normalize
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
otx = OTXv2("")

In [ ]:
indicators=otx.get_all_indicators(limit=100000,max_page=4)

In [ ]:
indicators=list(indicators)

In [ ]:
df = pd.DataFrame(indicators)

In [ ]:
print(len(df))

360361


In [ ]:
ipv4_indicators = df[df['type'] == "IPv4"]

In [ ]:
ipv4_indicators.head()

,id,indicator,type,created,content,title,description,expiration,is_active,role
0,20601,71.6.165.200,IPv4,2022-03-27T16:42:07,,CAS-W: TCP-Scanner. Port: 23 (3),,2022-07-19T00:00:00,1,scanning_host
1,32836,71.6.167.142,IPv4,2022-03-28T18:55:23,,LT: TCP-Scanner. Port: 22 (4),,2022-07-22T00:00:00,1,scanning_host
2,33527,222.73.56.10,IPv4,2022-03-21T21:17:40,,DIS: Brutforce ssh Login (5),,2022-07-16T00:00:00,1,bruteforce
3,33836,184.105.247.196,IPv4,2022-03-25T05:26:18,,GIZ-W: TCP-Scanner. Port: 5900 (2),,2022-07-14T00:00:00,1,scanning_host
4,33851,71.6.135.131,IPv4,2022-03-22T03:38:37,,GIZ-W: TCP-Scanner. Port: 23 (8),,2022-07-23T00:00:00,1,scanning_host


In [ ]:
Name_list = ipv4_indicators["indicator"].tolist()

In [ ]:
ipv4_indicators_uniq=list(set(Name_list))

In [ ]:
print(len(ipv4_indicators_uniq))

107992


group   events by indicators

In [ ]:
gk = ipv4_indicators.groupby('indicator')

In [ ]:
print(len(gk))

107992


In [ ]:
unq1=[0,0]
size=gk.size()
sz = size.to_frame()
sz = sz.reset_index(level=0)
selected_indi=sz. loc[sz[0] == 8]
# selected_indi.head()
# tt=selected_indi.iat[0,0]
# print(tt)
# print(type(tt))
unq1[0]=selected_indi.iat[0,0]
unq1[1]=selected_indi.iat[1,0]
print(unq1)

['1.15.175.127', '1.234.58.220']


In [ ]:
size[size>6]

indicator
1.12.255.18       7
1.14.69.182       7
1.15.175.127      7
1.204.71.210      7
1.214.245.27      7
                 ..
96.78.175.37      7
98.128.172.193    7
98.142.141.184    8
98.152.112.54     8
98.195.176.219    7
Length: 3765, dtype: int64

In [ ]:
# unq= ipv4_indicators_uniq
unq=['1.15.175.127','1.234.58.220']

In [ ]:
indicator_pulses=[]
import json
for i in tqdm_notebook(unq):
  #print(i)
  indi_info5=otx.get_indicator_details_full(IndicatorTypes.IPv4,i)
  json_object = json.dumps(indi_info5, indent = 4)
  final_dictionary = json.loads(json_object)
  pulse_ids=[]
  pulse_count=len(final_dictionary['general']['pulse_info']['pulses'])
  for j in range(pulse_count):
     pulse_ids.append(final_dictionary['general']['pulse_info']['pulses'][j]['id'])
  # print(i)
  a={}
  a={i:pulse_ids}
  indicator_pulses.append(a)
  # print(pulse_ids)
  # print(json_object)
  # print("--------")
print(indicator_pulses)


  0%|          | 0/2 [00:00<?, ?it/s]

[{'1.15.175.127': ['6237e03add4f862fc13dc7d3', '602bc528f447d628d41494f2', '60ece5998a5b54a5ffe75cb4', '621db66b1925e1de3361bff9', '622408868c0831c353d4901d', '622530119dde5708810bb5da', '6231ee27e6834a707de700ae', '624686ecf5d8cfa613a3c430', '624686ea4b644bef92992deb', '624f7b0bda7d6de7737d002e', '623aec53fe0e5289469ed60c', '623aec5035a3ac92a50307a2', '620d2b02cb060e1bd1aad8cc', '62327c7744d42b4525123814', '6230b93697f4af20515212fb', '620835fd721f345ba53eb906', '61fdbe0d540446ddc72b82cc', '61fbd7c91f2140ec427be980', '61f5b816ab1ca63a3f67712b', '60637f141cda4877a64d0872', '6196e4b4abff808deabdc776', '617381bbbefb2ec0927256d3', '6170debbc79a2f2386267bf9', '616cea3b78da308245afd8cb', '615f5a8314bf0f9120c50154', '615f5b503749c911868afe97', '615923bc51d45b884f136ebc', '614b9b69c9cddd5a296ce4ee', '5fef5b647be6e406857e1b59', '613479063b0e18560e948cd6', '613398073871f8e725534d85', '61062a651d9d095e4bdd9e45', '61062a6e2d64774370d534df', '611c4d88ead7b8e7e3b24a0c', '611afc02fd0dfedc6a6543c7', '

In [ ]:
print(len(indicator_pulses))

2


In [ ]:
kk={key: [] for key in unq}
print(kk)

{'1.15.175.127': [], '1.234.58.220': []}


In [ ]:
dic={}
for index in range(len(indicator_pulses)):
  for key in indicator_pulses[index]:
    puls_idss=indicator_pulses[index][key]
  for id in puls_idss:
    pulse_details = otx.get_pulse_details(id)
    ind=pulse_details['indicators']
    if ind:
      for i in ind:
        if i['indicator']==key:
          indi_info=i
          # kk[key].append(i)
          tag = pulse_details['tags']
          targeted_countries = pulse_details['targeted_countries']
          industries = pulse_details['industries']
          dict1 = {'tag':tag, 'targeted_countries':targeted_countries, 'industries':industries}
          indi_info.update(dict1)
          kk[key].append(indi_info)

          # print("yes")
  # print(ind)
print(kk)

{'1.15.175.127': [{'id': 2964745680, 'indicator': '1.15.175.127', 'type': 'IPv4', 'created': '2022-03-31T09:31:41', 'content': '', 'title': 'HHV: Brutforce ssh Login (4)', 'description': '', 'expiration': '2022-07-18T00:00:00', 'is_active': 1, 'tag': ['Brutforce', 'Webattack', 'smtp', 'ssh', 'scanning', 'tcp', 'Scanner'], 'targeted_countries': ['Germany'], 'industries': ['Serverhosting', 'Technology']}, {'id': 2964745680, 'indicator': '1.15.175.127', 'type': 'IPv4', 'created': '2022-04-01T10:31:24', 'content': '', 'title': 'ssh bruteforce attack (Lu)', 'description': '', 'expiration': '2022-05-01T10:00:00', 'is_active': 1, 'tag': ['SSH', 'scanner', 'attack', 'login', 'Telnet'], 'targeted_countries': [], 'industries': []}, {'id': 2964745680, 'indicator': '1.15.175.127', 'type': 'IPv4', 'created': '2022-03-16T18:24:57', 'content': '', 'title': 'SSH intrusion attempt from 1.15.175.127', 'description': '', 'expiration': '2022-04-15T18:00:00', 'is_active': 1, 'tag': ['Bruteforce', 'Brute-Fo

only for test


In [ ]:
dic={}
for index in range(len(indicator_pulses)):
  for key in indicator_pulses[index]:
    puls_idss=indicator_pulses[index][key]
  for id in puls_idss:
    pulse_details = otx.get_pulse_details(id)
    ind=pulse_details['indicators']
    if ind:
      for i in ind:
        if i['indicator']==key:
          indi_info=i
          # kk[key].append(i)
          tag = pulse_details['tags']
          targeted_countries = pulse_details['targeted_countries']
          industries = pulse_details['industries']
          dict1 = {'tag':tag, 'targeted_countries':targeted_countries, 'industries':industries}
          indi_info.update(dict1)
          indi_inf=get_role(i=indi_info,k=key,gk=gk)
          kk[key].append(indi_inf)

          # print("yes")
  # print(ind)
print(kk)

{'id': 2964745680, 'indicator': '1.15.175.127', 'type': 'IPv4', 'created': '2022-03-31T09:31:41', 'content': '', 'title': 'HHV: Brutforce ssh Login (4)', 'description': '', 'expiration': '2022-07-18T00:00:00', 'is_active': 1, 'tag': ['Brutforce', 'Webattack', 'smtp', 'ssh', 'scanning', 'tcp', 'Scanner'], 'targeted_countries': ['Germany'], 'industries': ['Serverhosting', 'Technology'], 'role': 'bruteforce'}
{'id': 2964745680, 'indicator': '1.15.175.127', 'type': 'IPv4', 'created': '2022-04-01T10:31:24', 'content': '', 'title': 'ssh bruteforce attack (Lu)', 'description': '', 'expiration': '2022-05-01T10:00:00', 'is_active': 1, 'tag': ['SSH', 'scanner', 'attack', 'login', 'Telnet'], 'targeted_countries': [], 'industries': [], 'role': 'bruteforce'}
{'id': 2964745680, 'indicator': '1.15.175.127', 'type': 'IPv4', 'created': '2022-03-16T18:24:57', 'content': '', 'title': 'SSH intrusion attempt from 1.15.175.127', 'description': '', 'expiration': '2022-04-15T18:00:00', 'is_active': 1, 'tag': 

In [ ]:
def get_role(i,k,gk):
  cr=i['created']
  gk_info=gk.get_group(k)
  eq=gk_info.loc[gk_info['created'] == cr]
  if len(eq)>0:
    role=eq.iloc[0]['role']
    dic1={'role':role}
    i.update(dic1)
  else:
    dic1={'role':None}
    i.update(dic1)
  print(i)
  return (i)

In [ ]:
for k in kk.keys():
  ind=kk[k]
  for i in ind:
    cr=i['created']
    gk_info=gk.get_group(k)
    eq=gk_info.loc[gk_info['created'] == cr]
    if len(eq)>0:
      role=eq.iloc[0]['role']
      dic1={'role':role}
      i.update(dic1)
    else:
      dic1={'role':None}
      i.update(dic1)

In [ ]:
print(kk)

{'98.142.141.184': [{'id': 2804418782, 'indicator': '98.142.141.184', 'type': 'IPv4', 'created': '2022-04-02T02:24:15', 'content': '', 'title': 'SSH intrusion attempt from 98.142.141.184', 'description': '', 'expiration': '2022-05-02T02:00:00', 'is_active': 1, 'tag': ['Bruteforce', 'Brute-Force', 'SSH', 'Honeypot'], 'targeted_countries': [], 'industries': [], 'role': 'bruteforce'}, {'id': 2804418782, 'indicator': '98.142.141.184', 'type': 'IPv4', 'created': '2022-03-22T06:35:02', 'content': '', 'title': 'PHI: Brutforce ssh Login (5)', 'description': '', 'expiration': '2022-07-11T00:00:00', 'is_active': 1, 'tag': ['Brutforce', 'Webattack', 'smtp', 'ssh', 'scanning', 'tcp', 'Scanner'], 'targeted_countries': ['Germany'], 'industries': ['Serverhosting', 'Technology'], 'role': 'bruteforce'}, {'id': 2804418782, 'indicator': '98.142.141.184', 'type': 'IPv4', 'created': '2022-03-14T22:00:12', 'content': '', 'title': 'HoneyNet Event:98.142.141.184', 'description': 'HoneyNet Event: 98.142.141.18

In [ ]:
industry=[]
for k in kk.keys():
  info=kk[k]
  for i in info:
    indu=i['industries']
    if len(indu)>0:
      for j in indu:
        industry.append(j)



In [ ]:
list_of_industries=list(set(industry))
print(list_of_industries)

['Serverhosting', 'Technology']


In [ ]:
gk.get_group('98.142.141.184')

,id,indicator,type,created,content,title,description,expiration,is_active,role
7208,2804418782,98.142.141.184,IPv4,2022-03-22T06:35:02,,PHI: Brutforce ssh Login (5),,2022-07-11T00:00:00,1,bruteforce
54545,2804418782,98.142.141.184,IPv4,2022-04-02T02:24:15,,SSH intrusion attempt from 98.142.141.184,,2022-05-02T02:00:00,1,bruteforce
94187,2804418782,98.142.141.184,IPv4,2022-03-14T22:00:13,,HoneyNet Event:98.142.141.184,HoneyNet Event: 98.142.141.184 connected: 7 ti...,2022-04-13T22:00:00,1,None
125743,2804418782,98.142.141.184,IPv4,2022-03-14T22:00:12,,HoneyNet Event:98.142.141.184,HoneyNet Event: 98.142.141.184 connected: 7 ti...,2022-04-13T22:00:00,1,None
168179,2804418782,98.142.141.184,IPv4,2022-03-28T04:00:09,,HoneyNet Event:98.142.141.184,HoneyNet Event: 98.142.141.184 connected: 83 t...,2022-04-27T04:00:00,1,None
207748,2804418782,98.142.141.184,IPv4,2022-03-14T22:00:17,,HoneyNet Event:98.142.141.184,HoneyNet Event: 98.142.141.184 connected: 7 ti...,2022-04-13T22:00:00,1,None
224334,2804418782,98.142.141.184,IPv4,2022-03-14T22:00:14,,HoneyNet Event:98.142.141.184,HoneyNet Event: 98.142.141.184 connected: 7 ti...,2022-04-13T22:00:00,1,None
287877,2804418782,98.142.141.184,IPv4,2022-04-07T00:00:14,,,SSH bruteforce authentication activity,2022-05-07T00:00:00,1,bruteforce


In [ ]:
import pandas as pd
import pandas as np

In [ ]:

# Import pandas library
import pandas as pd

# initialize list of lists
data = [['tom', None], ['nick', None], ['juli', None]]

# Create the pandas DataFrame
dff = pd.DataFrame(data, columns = ['Name', 'Age'])

# print dataframe.
dff

,Name,Age
0,tom,None
1,nick,None
2,juli,None


In [ ]:
se=dff.loc[dff['Name'] == 'yas']
print(type(se))
se
if len(se)==0:
  print("yes")

print(len(se))
# ss=se.iloc[0]['Age']
# print(ss)

<class 'pandas.core.frame.DataFrame'>
yes
0


In [ ]:
dic={'a':[]}
dic['a'].append(ss)
print(dic)
if dic['a'][0]==None:
  print("yes")

{'a': [None]}
yes


In [ ]:
print(kk)

In [ ]:
lstt={key: [] for key in unq}
print(lstt)

In [ ]:
for k in kk.keys():
  info=kk[k]
  # lst=[]
  for i in info:
    creat=i['created']
    # lst.append(creat)
    lstt[k].append(creat)
print(lstt)

In [ ]:
pip install tqdm

In [ ]:
from tqdm.notebook import tqdm_notebook

In [ ]:
a=['a','b','d','d','h','m','y']

for i in tqdm_notebook(a):
  print(i)
